#Tratando os dados NIST

Primeiramente vamos ler o vcf fornecido e salvar em um dataframe.

In [ ]:
#unzipar
!gunzip /content/NIST.vcf.gz

In [2]:
import pandas as pd

# Caminho para o seu arquivo .vcf
vcf_file = 'NIST.vcf'

# Abrir o arquivo e localizar a linha do cabeçalho
with open(vcf_file, 'r') as file:
    for i, line in enumerate(file):
        if line.startswith('#CHROM'):
            header_line = i
            break

# Agora leia o arquivo a partir da linha do cabeçalho
vcf_df = pd.read_csv(vcf_file, sep='\t', skiprows=header_line)

# Exibir as primeiras linhas do DataFrame
vcf_df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NIST-hg001-7001
0,1,324822,.,A,T,2639.77,GATKStandardMQ,AC=2;AF=1.00;AN=2;DP=152;Dels=0.00;FS=0.000;GC...,GT:AD:DP:GQ:PL,"1/1:37,115:152:99:2668,214,0"
1,1,664834,rs377593965,T,G,554.77,GATKStandardMQ,AC=1;AF=0.500;AN=2;BaseQRankSum=-0.297;DB;DP=6...,GT:AD:DP:GQ:PL,"0/1:18,48:66:99:583,0,134"
2,1,762273,rs3115849,G,A,1692.77,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=-0.377;DB;DP=81...,GT:AD:DP:GQ:PL,"1/1:1,80:81:92:1721,92,0"
3,1,877831,rs6672356,T,C,240.84,PASS,AC=2;AF=1.00;AN=2;DB;DP=12;Dels=0.00;FS=0.000;...,GT:AD:DP:GQ:PL,"1/1:0,12:12:18:269,18,0"
4,1,881627,rs2272757,G,A,557.77,PASS,AC=2;AF=1.00;AN=2;DB;DP=23;Dels=0.00;FS=0.000;...,GT:AD:DP:GQ:PL,"1/1:0,23:23:42:586,42,0"


Agora vamos filtrar apenas as linhas que possuem o PASS em FILTER, que indicam que passaram na análise dos pipelines anteriores.

In [3]:
vcf_df = vcf_df.loc[vcf_df.FILTER ==  'PASS']

Como eu preciso pesquisar os dbSNP ID (rsID), vou focar apenas nos dados que apresnetam um ID.

In [4]:
vcf_df = vcf_df[vcf_df.ID != '.']

In [5]:
vcf_df.head(5)

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NIST-hg001-7001
2,1,762273,rs3115849,G,A,1692.77,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=-0.377;DB;DP=81...,GT:AD:DP:GQ:PL,"1/1:1,80:81:92:1721,92,0"
3,1,877831,rs6672356,T,C,240.84,PASS,AC=2;AF=1.00;AN=2;DB;DP=12;Dels=0.00;FS=0.000;...,GT:AD:DP:GQ:PL,"1/1:0,12:12:18:269,18,0"
4,1,881627,rs2272757,G,A,557.77,PASS,AC=2;AF=1.00;AN=2;DB;DP=23;Dels=0.00;FS=0.000;...,GT:AD:DP:GQ:PL,"1/1:0,23:23:42:586,42,0"
5,1,883899,rs72631890,T,G,268.77,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=3.501;DB;DP=31...,GT:AD:DP:GQ:PL,"0/1:15,16:31:99:297,0,302"
6,1,887801,rs3828047,A,G,531.77,PASS,AC=2;AF=1.00;AN=2;DB;DP=18;Dels=0.00;FS=0.000;...,GT:AD:DP:GQ:PL,"1/1:0,18:18:42:560,42,0"


#Banco de dados de SNPS.

Provavelmente existem maneiras mais faceis de realizar esse processo, como por exemplo utilizando APIs. Mas eu não tive tempo de pesquisar em como utiliza-las para esse desafio. Logo, eu pensei que a melhor estratégia fosse utilizar um arquivo VCF contendo as informações de SNPs de um banco de dados confiaveisl


O https://ftp.ncbi.nih.gov/snp/organisms/human_9606/VCF/ apresenta diversos tipos de .VCF ontendo os SNPS. O mais completo seria 00-All.vcf.gz, entretando ele possui 15 GB zipado. Eu não tinha memória para lhe dar com tamanho dado, então eu utilizei o common_all_20180418.vcf.gz.

Logo, para a aplicação, será necessário a importação desse arquivo como um banco de dados.

In [6]:
#baixando o banco de dados
!wget https://ftp.ncbi.nih.gov/snp/organisms/human_9606/VCF/common_all_20180418.vcf.gz

--2024-09-14 13:27:39--  https://ftp.ncbi.nih.gov/snp/organisms/human_9606/VCF/common_all_20180418.vcf.gz
Resolving ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)... 130.14.250.13, 130.14.250.10, 130.14.250.11, ...
Connecting to ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1592589437 (1.5G) [application/x-gzip]
Saving to: ‘common_all_20180418.vcf.gz’

common_all_20180418 100%[===================>]   1.48G  24.5MB/s    in 82s     

2024-09-14 13:29:01 (18.5 MB/s) - ‘common_all_20180418.vcf.gz’ saved [1592589437/1592589437]



In [7]:
#unzip
!gunzip common_all_20180418.vcf.gz

#Comparando os arquivo VCF para retirar as informações necessárias.

Vou realizar esse processo utiilizando chunks para ler os arquivos grandes em partes.

In [58]:
import pandas as pd

# Caminho para o arquivo VCF
vcf_file = '/content/common_all_20180418.vcf'


ids_from_vcf = list(vcf_df.ID)

# Exemplo de lista de IDs que você quer filtrar (converter para set)
ids_from_vcf = set(ids_from_vcf)  # Exemplo, substitua pelos IDs reais
#print(ids_from_vcf)

# Função otimizada para processar o arquivo em chunks e filtrar as linhas com IDs específicos
def process_vcf_with_filter(vcf_file, ids_from_vcf, chunk_size=50000):
    filtered_data = []  # Lista para armazenar as linhas filtradas

    with open(vcf_file, 'r') as f:
        for line in f:
            if not line.startswith('#'):  # Ignorar cabeçalhos
                columns = line.strip().split('\t')  # Dividir por tabulação
                # Verificar se o ID da linha está no set de IDs
                if columns[2] in ids_from_vcf:  # A coluna 2 é o ID no formato VCF
                    filtered_data.append(columns)

            # Processar o chunk quando atingir o tamanho definido
            if len(filtered_data) >= chunk_size:
                # Criar o DataFrame a partir do chunk filtrado
                vcf_filtered_df = pd.DataFrame(filtered_data, columns=["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"])
                # Aqui você pode salvar ou processar o DataFrame filtrado
                #print(vcf_filtered_df.head())  # Exemplo: imprimir as primeiras linhas do chunk filtrado
                filtered_data = []  # Limpar o chunk

        # Processar o último chunk (caso haja linhas restantes)
        if filtered_data:
            vcf_filtered_df = pd.DataFrame(filtered_data, columns=["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"])
            #print(vcf_filtered_df.head())  # Exemplo: imprimir as primeiras linhas do último chunk

    return vcf_filtered_df

# Chamar a função com a lista de IDs e um tamanho de chunk maior
filtered_vcf_df = process_vcf_with_filter(vcf_file, ids_from_vcf, chunk_size=50000)

# Exibir o DataFrame final com os dados filtrados
filtered_vcf_df.head(5)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,1,826893,rs3115849,G,A,.,.,RS=3115849;RSPOS=826893;RV;dbSNPBuildID=103;SS...
1,1,946247,rs2272757,G,A,.,.,RS=2272757;RSPOS=946247;RV;dbSNPBuildID=100;SS...
2,1,952421,rs3828047,A,"G,T",.,.,RS=3828047;RSPOS=952421;dbSNPBuildID=107;SSR=0...
3,1,953259,rs3748596,T,C,.,.,RS=3748596;RSPOS=953259;dbSNPBuildID=107;SSR=0...
4,1,953279,rs3748597,T,C,.,.,RS=3748597;RSPOS=953279;dbSNPBuildID=107;SSR=0...


Aqui podemos ver que não foi possível identificar todos os rsID do banco de dados escolhido. Acredito que se fosse o all_snps eu conseguiria toda a cobertura. Mas isso é modulavel, como eu escolho o banco de dados em forma de VCF, eu posso rodar essa análise contra qualquer banco padronizado

In [10]:
print('Ids encontrados:', len(filtered_vcf_df))
print('total de IDs', len(ids_from_vcf))

Ids encontrados: 22763
total de IDs 25776


In [40]:
filtered_vcf_df.head(5)

,CHROM,POS,ID,REF,ALT,TOPMED,GENEINFO,VC
0,1,826893,rs3115849,G,A,NaN,LINC01128:643837|LINC00115:79854,SNV
1,1,946247,rs2272757,G,A,NaN,NOC2L:26155,SNV
2,1,952421,rs3828047,A,"G,T",NaN,NOC2L:26155,SNV
3,1,953259,rs3748596,T,C,NaN,NOC2L:26155,SNV
4,1,953279,rs3748597,T,C,NaN,NOC2L:26155,SNV


Agora é só extrair as informações do meu interesse por meio de funções e regex.

In [12]:
def extract_topmed(info):
    if 'TOPMED=' in info:
        return info.split('TOPMED=')[1].split(';')[0]
    else:
        return 0


def extract_geneinfo(info):
    if 'GENEINFO=' in info:
        return info.split('GENEINFO=')[1].split(';')[0]
    else:
        return 0


def extract_vc(info):
    if 'VC=' in info:
        return info.split('VC=')[1].split(';')[0]
    else:
        return 0


def extract_dp(info):
    if 'DP=' in info:
        return info.split('DP=')[1].split(';')[0]
    else:
        return None


In [52]:
# Aplicar as funções na coluna INFO
filtered_vcf_df['TOPMED'] = filtered_vcf_df['INFO'].apply(
    lambda x: extract_topmed(x))
filtered_vcf_df['GENEINFO'] = filtered_vcf_df['INFO'].apply(
    lambda x: extract_geneinfo(x))
filtered_vcf_df['VC'] = filtered_vcf_df['INFO'].apply(
    lambda x: extract_vc(x))

In [53]:
# prompt: remova filtered_vcf_df.pop(['INFO', 'QUAL', 'FILTER'])

filtered_vcf_df = filtered_vcf_df.drop(['INFO', 'QUAL', 'FILTER'], axis=1)

In [44]:
filtered_vcf_df.head()

,CHROM,POS,ID,REF,ALT,TOPMED,GENEINFO,VC
0,1,826893,rs3115849,G,A,"0.27686990316004077,0.72313009683995922",LINC01128:643837|LINC00115:79854,SNV
1,1,946247,rs2272757,G,A,"0.52584257135575942,0.47415742864424057",NOC2L:26155,SNV
2,1,952421,rs3828047,A,"G,T","0.05807212028542303,0.93134397298674821,0.0105...",NOC2L:26155,SNV
3,1,953259,rs3748596,T,C,"0.06854453363914373,0.93145546636085626",NOC2L:26155,SNV
4,1,953279,rs3748597,T,C,"0.06855249745158002,0.93144750254841997",NOC2L:26155,SNV


Agora as frequencias do TOPMED precisam ser tratadas. A lógica aqui é a seguinte:

- ir no meu arquivo NIST.vcf e identificar o SNP
- Ver qual é esse SNP e sua respectiva freq. no TOPMED
- Escolher ela e salvar

In [45]:
import numpy as np

def format_topmed(topmed_str):
        if pd.isnull(topmed_str):
            return ""
        topmed_str = str(topmed_str)
        topmed_list = topmed_str.split(',')
        formatted_list = [s[0:4] for s in topmed_list]
        return ','.join(formatted_list)

# Definindo a função para atualizar a coluna TOPMED

def update_topmed(row, freq_data):
    id = row['ID']
    alt = row['ALT']

    try:
        # Tenta acessar o valor correspondente
        freq = freq_data[id][alt]
    except KeyError:
        # Caso não encontre a chave, define um valor padrão
        freq = '.'

    # Retorna o valor formatado
    return freq


# Função para converter os valores

def convert_to_float(value):
    return float(value) if value != '.' else np.nan

In [54]:
filtered_vcf_df['TOPMED'] = filtered_vcf_df['TOPMED'].apply(format_topmed)

freq_data = {}

for a, b, c, d in zip(filtered_vcf_df['ID'], filtered_vcf_df['REF'], filtered_vcf_df['ALT'], filtered_vcf_df['TOPMED']):

    bases = b
    c = c.split(',')

    for e in c:
        bases = bases + ' ' + e

    bases = bases.split()


    d = str(d)

    k = d.split(',')

    TOPMED_freq_dic = {}

    for base, freq in zip(bases, k):
        TOPMED_freq_dic[base] = freq

    freq_data[a] = TOPMED_freq_dic

# dentro do meu dicionario o PRIMEIRO é o REF e depois são os ALT
# Aplicar a função para atualizar a coluna TOPMED com base na FREQ ALT do NIST
filtered_vcf_df['TOPMED'] = filtered_vcf_df.apply(
    lambda row: update_topmed(row, freq_data), axis=1)

# Aplicando a conversão à coluna TOPMED
filtered_vcf_df['TOPMED'] = filtered_vcf_df['TOPMED'].apply(
    convert_to_float)


In [55]:
filtered_vcf_df.head(3)

,CHROM,POS,ID,REF,ALT,TOPMED,GENEINFO,VC
0,1,826893,rs3115849,G,A,0.72,LINC01128:643837|LINC00115:79854,SNV
1,1,946247,rs2272757,G,A,0.47,NOC2L:26155,SNV
2,1,952421,rs3828047,A,"G,T",NaN,NOC2L:26155,SNV


Vamos adicionar os genes Names a partir dessa função e utilizando o banco de dados https://www.ncbi.nlm.nih.gov/datasets/gene/GCF_000001405.40/:


In [ ]:
# prompt: leia o ncbi_human_sapiens_genes.tsv em um df.

import pandas as pd
# Leia o arquivo ncbi_human_sapiens_genes.tsv em um DataFrame
df_genes = pd.read_csv('/content/ncbi_human_sapiens_genes.tsv', sep='\t')

# Exiba as primeiras linhas do DataFrame para verificar se a leitura foi bem-sucedida
df_genes.head()


,Accession,Begin,End,Chromosome,Orientation,Name,Symbol,Gene ID,Gene Type,Transcripts accession,Protein accession,Protein length,Locus tag
0,NC_000001.11,11874,14409,1,plus,DEAD/H-box helicase 11 like 1 (pseudogene),DDX11L1,100287102,transcribed_pseudogene,NR_046018.2,NaN,NaN,NaN
1,NC_000001.11,14362,29370,1,minus,"WASP family homolog 7, pseudogene",WASH7P,653635,transcribed_pseudogene,NR_024540.1,NaN,NaN,NaN
2,NC_000001.11,17369,17436,1,minus,microRNA 6859-1,MIR6859-1,102466751,miRNA,NR_106918.1,NaN,NaN,NaN
3,NC_000001.11,29774,35418,1,plus,MIR1302-2 host gene,MIR1302-2HG,107985730,lncRNA,XR_007065314.1,NaN,NaN,NaN
4,NC_000001.11,30366,30503,1,plus,microRNA 1302-2,MIR1302-2,100302278,miRNA,NR_036051.1,NaN,NaN,NaN


In [ ]:
import pandas as pd

# Função para extrair os símbolos dos genes da coluna GENEINFO
def extract_symbols(geneinfo):
    symbols = []
    for gene in geneinfo.split('|'):  # Separando por '|' quando houver mais de um gene
        symbol = gene.split(':')[0]  # Pegando o símbolo antes de ':'
        symbols.append(symbol)
    return symbols

# Aplicar a função na coluna GENEINFO para extrair os símbolos
df['Genes'] = df['GENEINFO'].apply(extract_symbols)

# Função para buscar o nome correspondente ao símbolo no df_genes
def get_gene_names(symbols):
    names = []
    for symbol in symbols:
        # Procurar o nome no df_genes usando o símbolo
        name = df_genes[df_genes['Symbol'] == symbol]['Name'].values
        if len(name) > 0:
            names.append(name[0])  # Adicionar o nome correspondente
        else:
            names.append('NaN')  # Caso o símbolo não seja encontrado, adicionar 'NaN'
    return '|'.join(names)  # Retornar nomes separados por '|', se houver mais de um


# Leia o arquivo ncbi_human_sapiens_genes.tsv em um DataFrame
df_genes = pd.read_csv('/content/ncbi_human_sapiens_genes.tsv', sep='\t')


# Aplicar a função para obter os nomes e criar a nova coluna 'Gene Names'
df['Gene Names'] = df['Genes'].apply(get_gene_names)

# Excluir a coluna GENEINFO
df.drop(columns=['GENEINFO'], inplace=True)

# Exibir o dataframe atualizado
df

,CHROM,POS,ID,REF,ALT,TOPMED,VC,DP,Genes,Gene Names
0,1,826893,rs3115849,G,A,0.72,SNV,81,"[LINC01128, LINC00115]",long intergenic non-protein coding RNA 1128|lo...
1,1,946247,rs2272757,G,A,0.47,SNV,23,[NOC2L],NOC2 like nucleolar associated transcriptional...
2,1,952421,rs3828047,A,"G,T",.,SNV,18,[NOC2L],NOC2 like nucleolar associated transcriptional...
3,1,953259,rs3748596,T,C,0.93,SNV,39,[NOC2L],NOC2 like nucleolar associated transcriptional...
4,1,953279,rs3748597,T,C,0.93,SNV,34,[NOC2L],NOC2 like nucleolar associated transcriptional...
...,...,...,...,...,...,...,...,...,...,...
22758,X,154019032,rs1059701,G,A,0.57,SNV,12,[IRAK1],interleukin 1 receptor associated kinase 1
22759,X,154153068,rs949431,G,T,.,SNV,227,[OPN1LW],"opsin 1, long wave sensitive"
22760,X,154428737,rs28497482,G,A,0.09,SNV,8,"[ATP6AP1, CH17-340M24.3]",ATPase H+ transporting accessory protein 1|NaN
22761,X,154766321,rs2728532,G,"A,T",.,SNV,76,[DKC1],dyskerin pseudouridine synthase 1


Agora o resto é o FLASK com filtro e uma estilização por bootstrap.